First updated @ Nov 2, 2021

Last updated @ Nov 2, 2021

Data Source: https://www.kaggle.com/rounakbanik/the-movies-dataset?select=credits.csv

I downloaded the data from Kaggle directly and now we need to clean up this mess.

In [1]:
import pandas as pd
import json
import ast
from collections import defaultdict

In [2]:
# load data, we only care about the cast and crew for now
# I am putting TheMoviesData in the same directory as the Data directory
# I am not uploading this data to GitHub because it is too big, but you can download on Kaggle freely
df = pd.read_csv('../TheMoviesData/credits.csv')
df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [3]:
# check if there are any empty values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


## Counting

The info in cast and crew is very disorganized. We only want the gender information, so we need to loop through every cast and crew and count their gender. 

Stringified JSON object to list: https://stackoverflow.com/questions/42631097/convert-stringified-list-of-dictionaries-back-to-a-list-of-dictionaries

In [12]:
df_new = pd.DataFrame(columns=[
    'id', 
    'cast_num', 
    'cast_female_num', 
    'cast_male_num', 
    'cast_unknown_num',
    'crew_num', 
    'crew_female_num', 
    'crew_male_num',
    'crew_unknown_num',
    'Directing_total', 
    'Directing_female',
    'Directing_male',
    'Directing_unknown',
    'Writing_total',
    'Writing_female',
    'Writing_male',
    'Writing_unknown',
    'Production_total',
    'Production_female',
    'Production_male',
    'Production_unknown',
    ])
df_new['id'] = df['id']
df_new.head()

,id,cast_num,cast_female_num,cast_male_num,cast_unknown_num,crew_num,crew_female_num,crew_male_num,crew_unknown_num,Directing_total,...,Directing_male,Directing_unknown,Writing_total,Writing_female,Writing_male,Writing_unknown,Production_total,Production_female,Production_male,Production_unknown
0,862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
overview_info = defaultdict(int)

for index, row in df[:20].iterrows():
    cast_info, crew_info, crew_department = defaultdict(int), defaultdict(int), defaultdict(int)
    cast_list = ast.literal_eval(row['cast'])
    crew_list = ast.literal_eval(row['crew'])

    # count everything in cast
    for cast in cast_list:
        if cast['gender'] == 2:
            cast_info['male'] += 1
        elif cast['gender'] == 1:
            cast_info['female'] += 1
        elif cast['gender'] == 0:
            cast_info['unknown'] += 1
        else:
            print(f'Oops1 index{index} is unusal: {cast}')
    df_new.at[index, 'cast_num'] = len(cast_list)
    df_new.at[index, 'cast_female_num'] = cast_info['female']
    df_new.at[index, 'cast_male_num'] = cast_info['male']
    df_new.at[index, 'cast_unknown_num'] = cast_info['unknown']

    # count evetything in crew
    for crew in crew_list:
        if crew['gender'] == 2:
            crew_info['male'] += 1
            crew_department[str(crew['department'])+'_male'] += 1
            crew_department[str(crew['department'])+'_total'] += 1
        elif crew['gender'] == 1:
            crew_info['female'] += 1
            crew_department[str(crew['department'])+'_female'] += 1
            crew_department[str(crew['department'])+'_total'] += 1
        elif crew['gender'] == 0:
            crew_info['unknown'] += 1
            crew_department[str(crew['department'])+'_unknown'] += 1
            crew_department[str(crew['department'])+'_total'] += 1
        else:
            print(f'Oops2 index{index} is unusal: {crew}')
    df_new.at[index, 'crew_num'] = len(crew_list)
    df_new.at[index, 'crew_female_num'] = crew_info['female']
    df_new.at[index, 'crew_male_num'] = crew_info['male']
    df_new.at[index, 'crew_unknown_num'] = crew_info['unknown']

    # aggregate all counts to overview data
    overview_info['cast_total'] += len(cast_list)
    for cast_gender, count in cast_info.items():
        overview_info['cast_'+str(cast_gender)] += count
    overview_info['crew_total'] += len(crew_list)
    for crew_gender, count in crew_info.items():
        overview_info['crew_'+str(crew_gender)] += count
    for department, count in crew_department.items():
        overview_info[department] += count
        df_new.at[index, department] = count

df_new.head()

,id,cast_num,cast_female_num,cast_male_num,cast_unknown_num,crew_num,crew_female_num,crew_male_num,crew_unknown_num,Directing_total,...,Sound_female,Camera_male,Camera_total,Art_female,Costume & Make-Up_female,Costume & Make-Up_total,Costume & Make-Up_unknown,Costume & Make-Up_male,Camera_unknown,Editing_female
0,862,13,3,9,1,106,4,29,73,3,...,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8844,26,6,9,11,16,0,14,2,1,...,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15602,7,3,4,0,4,0,4,0,1,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31357,10,4,6,0,10,1,3,6,1,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11862,12,5,6,1,7,2,5,0,1,...,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df_new.iloc[0]

id                           862.0
cast_num                      13.0
cast_female_num                3.0
cast_male_num                  9.0
cast_unknown_num               1.0
crew_num                     106.0
crew_female_num                4.0
crew_male_num                 29.0
crew_unknown_num              73.0
Directing_total                3.0
Directing_female               0.0
Directing_male                 1.0
Directing_unknown              2.0
Writing_total                  8.0
Writing_female                 0.0
Writing_male                   6.0
Writing_unknown                2.0
Production_total               9.0
Production_female              2.0
Production_male                2.0
Production_unknown             5.0
Editing_male                   2.0
Editing_total                 11.0
Art_male                       1.0
Art_total                      7.0
Sound_unknown                  8.0
Sound_total                   16.0
Visual Effects_unknown        36.0
Visual Effects_total

In [43]:
df_new = df_new.fillna(0) # replace all the NaN with 0
df_new.head()

,id,cast_num,cast_female_num,cast_male_num,cast_unknown_num,crew_num,crew_female_num,crew_male_num,crew_unknown_num,Directing_total,...,Sound_female,Camera_male,Camera_total,Art_female,Costume & Make-Up_female,Costume & Make-Up_total,Costume & Make-Up_unknown,Costume & Make-Up_male,Camera_unknown,Editing_female
0,862,13,3,9,1,106,4,29,73,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8844,26,6,9,11,16,0,14,2,1,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15602,7,3,4,0,4,0,4,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31357,10,4,6,0,10,1,3,6,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11862,12,5,6,1,7,2,5,0,1,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Too many Unknowns

In [44]:
overview_info

defaultdict(int,
            {'cast_total': 446,
             'cast_male': 240,
             'cast_female': 122,
             'cast_unknown': 84,
             'crew_total': 540,
             'crew_male': 222,
             'crew_unknown': 266,
             'crew_female': 52,
             'Directing_male': 23,
             'Directing_total': 33,
             'Writing_male': 41,
             'Writing_total': 57,
             'Writing_unknown': 9,
             'Production_female': 21,
             'Production_total': 101,
             'Production_unknown': 31,
             'Production_male': 49,
             'Editing_male': 17,
             'Editing_total': 48,
             'Art_male': 15,
             'Art_total': 47,
             'Sound_unknown': 43,
             'Sound_total': 76,
             'Visual Effects_unknown': 46,
             'Visual Effects_total': 54,
             'Sound_male': 31,
             'Editing_unknown': 28,
             'Crew_unknown': 32,
             'Crew_total'

In [45]:
print('Percentage of unknowns in Cast:', overview_info['cast_unknown']/overview_info['cast_total'])
print('Percentage of unknowns in Crew:', overview_info['crew_unknown']/overview_info['crew_total'])
print('Percentage of unknowns in Directing:', overview_info['Directing_unknown']/overview_info['Directing_total'])
print('Percentage of unknowns in Writing:', overview_info['Writing_unknown']/overview_info['Writing_total'])
print('Percentage of unknowns in Production:', overview_info['Production_unknown']/overview_info['Production_total'])

Percentage of unknowns in Cast: 0.18834080717488788
Percentage of unknowns in Crew: 0.4925925925925926
Percentage of unknowns in Directing: 0.21212121212121213
Percentage of unknowns in Writing: 0.15789473684210525
Percentage of unknowns in Production: 0.3069306930693069


We are not going to keep going down this road because we are only interested in 9,300+ of these 45,000 movies. I am not going to deciper meaningless movies.

In [38]:
overview_info['cast_unknown']

12

https://pypi.org/project/gender-guesser/

In [46]:
%pip install gender-guesser

     |████████████████████████████████| 379 kB 617 kB/s 
Note: you may need to restart the kernel to use updated packages.


In [47]:
import gender_guesser.detector as gender
d = gender.Detector()

In [59]:
print(d.get_gender(u"Alison"))

female
